In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Cargar el archivo CSV
# Asegúrate de que el nombre del archivo coincida con el que subiste.
file_path = "data/prueba modelo cuadrante vehiculo.xlsx"
df = pd.read_excel(file_path)

# Renombrar columnas para facilitar el manejo (basado en el snippet de datos)
df.columns = [
    'CUADRANTE',
    'MES_N',
    'TOTAL_ROBOVe_MES_N',
    'TOTAL_ROBOVe_MES_N_MENOS_1',
    'TOTAL_ROBOVe_MES_N_MENOS_2'
]

# Mostrar las primeras filas
print("### 1. Datos Originales (Primeras 5 Filas) ###")
print(df.head())
print("-" * 50)

FileNotFoundError: [Errno 2] No such file or directory: 'data/prueba modelo cuadrante vehiculo.xlsx'

In [ ]:
# Reestructurar los datos para el modelo de predicción de 78 salidas
# Vamos a agrupar los datos por MES_N.

# La 'Y' (salida) serán los 78 robos del Mes N, apilados en un array.
Y_data = df.groupby('MES_N')['TOTAL_ROBOVe_MES_N'].apply(lambda x: x.values).tolist()
Y = np.array(Y_data)

# Las 'X' (entradas) serán los 78 cuadrantes, el mes N-1 y el mes N-2.
# En este caso, usaremos el CUADRANTE, N-1 y N-2 para predecir N.
X_features = df[['CUADRANTE', 'TOTAL_ROBOVe_MES_N_MENOS_1', 'TOTAL_ROBOVe_MES_N_MENOS_2']]
X_data = X_features.groupby(df['MES_N']).apply(lambda x: x.values.flatten()).tolist()
X = np.array(X_data)

# Ajuste: La primera fila (MES_N = 1) no tiene datos N-1 ni N-2 válidos para todas las columnas,
# aunque ya está manejado con el snippet, generalmente se excluyen las filas iniciales
# que tienen datos N-1 y N-2 como 0, si estos no son ceros reales.
# Pero Keras lo puede manejar. Excluimos la primera fila (Mes 1) para evitar el problema de N-1 y N-2.
X = X[1:]
Y = Y[1:]

# Definir las dimensiones
input_dim = X.shape[1]  # 78 * 3 = 234 (variables de entrada)
output_dim = Y.shape[1] # 78 (variables de salida)

# Dividir para entrenamiento y prueba (usaremos el último mes para probar)
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.15, shuffle=False # Usamos shuffle=False para mantener el orden temporal
)

print(f"Dimensión de Entrada (X): {X.shape}")
print(f"Dimensión de Salida (Y): {Y.shape}")
print("-" * 50)

In [ ]:
# 1. Construir el modelo Keras
model = Sequential([
    # Capa de entrada (tamaño de la capa = Dimensión de entrada * 2, por ejemplo)
    Dense(256, activation='relu', input_shape=(input_dim,)),
    Dropout(0.2), # Regularización para evitar sobreajuste

    # Capa oculta
    Dense(128, activation='relu'),
    Dropout(0.2),

    # Capa de salida (debe tener 78 neuronas, una para cada cuadrante)
    Dense(output_dim, activation='linear') # 'linear' para problemas de regresión
])

# 2. Compilar el modelo
# Usamos 'mean_squared_error' (MSE) como función de pérdida, estándar en regresión.
# Optimizador Adam es eficiente.
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

print("### 2. Estructura del Modelo Keras ###")
model.summary()
print("-" * 50)

# 3. Entrenar el modelo
# Usamos EarlyStopping para detener el entrenamiento cuando el modelo deje de mejorar
# y evitar sobreajuste.
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

print("### 3. Entrenamiento del Modelo (puede tardar un momento) ###")
history = model.fit(
    X_train, Y_train,
    epochs=100,
    batch_size=4,
    validation_data=(X_test, Y_test),
    callbacks=[early_stop],
    verbose=0 # 0 para no mostrar el progreso en cada época
)
print("Entrenamiento finalizado. Mejores pesos restaurados.")
print("-" * 50)

In [ ]:
# 1. Generar Predicciones en el conjunto de prueba
Y_pred_test = model.predict(X_test)

# 2. Calcular R^2
# Para R^2 en un modelo de múltiples salidas, se compara el valor real Y_test
# (aplanado) con el valor predicho Y_pred_test (aplanado).
r2 = r2_score(Y_test.flatten(), Y_pred_test.flatten())

print(f"### 4. Evaluación del Modelo ###")
print(f"Coeficiente de Determinación (R^2): {r2:.4f}")

if r2 > 0.7:
    print("✅ ¡El modelo tiene una precisión alta!")
elif r2 > 0.5:
    print("⚠️ Precisión moderada. Podría requerir más capas o datos.")
else:
    print("❌ Precisión baja. Se recomienda ajustar hiperparámetros o la estructura del modelo.")

print("-" * 50)

In [ ]:
# 1. Des-estructurar las predicciones y re-insertarlas en el DataFrame

# Asumimos que Y_test corresponde a los últimos meses del DataFrame original (df).
# El tamaño de Y_pred_test es el número de meses en la prueba * 78 cuadrantes.

# El índice de inicio en el DataFrame original es donde comienzan los datos de prueba
start_index = len(df) - len(Y_test.flatten())

# Crear la columna de Predicción (inicialmente NaN para los datos de entrenamiento)
df['PREDICCION_MES_N'] = np.nan

# Insertar las predicciones en la sección de prueba del DataFrame
df.loc[start_index:, 'PREDICCION_MES_N'] = Y_pred_test.flatten()

print("### 5. DataFrame con la Columna de Predicción ###")
print(df.tail(80)) # Mostramos los últimos meses (donde hay predicciones)
print("-" * 50)

In [ ]:
# 2. Identificar los 5 Cuadrantes de Mayor Incidencia para el mes a predecir
# La última predicción es la que corresponde al 'Mes N' más alto que predijo
# (que es el último mes de prueba en este caso).

# Obtenemos el último conjunto de 78 predicciones.
ultimas_predicciones = df['PREDICCION_MES_N'].tail(78).to_frame()
ultimas_predicciones['CUADRANTE'] = df['CUADRANTE'].tail(78).values
ultimas_predicciones.columns = ['PREDICCION_ROBOS', 'CUADRANTE']

# Ordenar por predicción descendente
top_cuadrantes = ultimas_predicciones.sort_values(by='PREDICCION_ROBOS', ascending=False).head(5)

print("### 6. TOP 5 Cuadrantes con Mayor Incidencia Delictiva (Mes Predicho) ###")
print(top_cuadrantes)